# Análisis de Congruencia/Incongruencia Ideológica

Este notebook analiza los cambios de opinión (CO) y tiempo (CT) según su **congruencia ideológica**.

## Definiciones:

### **CONGRUENTES** (ideológicamente consistentes):
- Ítems **Progresistas** + movimiento hacia **Izquierda**
- Ítems **Conservadores** + movimiento hacia **Derecha**

### **INCONGRUENTES** (ideológicamente inconsistentes):
- Ítems **Progresistas** + movimiento hacia **Derecha**
- Ítems **Conservadores** + movimiento hacia **Izquierda**

## Análisis:

1. Crear variables de Congruencia/Incongruencia para CO y CT
2. Comparar Congruente vs Incongruente (todas las poblaciones)
3. Comparar por cada categoría (Categoria_PASO_2023)
4. Para Generales y Ballotage
5. Tablas Excel con resultados

In [1]:
import pandas as pd
import numpy as np
import os
from scipy import stats
from scipy.stats import wilcoxon
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

print("✓ Librerías cargadas exitosamente")

✓ Librerías cargadas exitosamente


## 1. Cargar Datos

In [2]:
# Rutas a los archivos Excel
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Bases definitivas')
Excel_Generales = os.path.join(Ruta_Base, 'Generales.xlsx')
Excel_Ballotage = os.path.join(Ruta_Base, 'Ballotage.xlsx')

# Cargar DataFrames desde Excel
df_Generales = pd.read_excel(Excel_Generales)
df_Ballotage = pd.read_excel(Excel_Ballotage)

dfs_Finales = {
    'Generales': df_Generales,
    'Ballotage': df_Ballotage
}

print(f"✓ Datos cargados desde Excel:")
print(f"  - Generales: {len(df_Generales)} registros")
print(f"  - Ballotage: {len(df_Ballotage)} registros")

✓ Datos cargados desde Excel:
  - Generales: 2786 registros
  - Ballotage: 1254 registros


## 2. Verificar Variables Necesarias

In [3]:
# Variables necesarias para CO
vars_necesarias_co = [
    'Cambio_Op_Sum_Pro_Izq',
    'Cambio_Op_Sum_Pro_Der',
    'Cambio_Op_Sum_Con_Izq',
    'Cambio_Op_Sum_Con_Der'
]

# Variables necesarias para CT
vars_necesarias_ct = [
    'Cambio_Tiempo_Sum_Pro_Izq',
    'Cambio_Tiempo_Sum_Pro_Der',
    'Cambio_Tiempo_Sum_Con_Izq',
    'Cambio_Tiempo_Sum_Con_Der'
]

print("Verificando variables necesarias:\n")

for nombre_df, df in dfs_Finales.items():
    print(f"{nombre_df}:")
    
    # Verificar CO
    faltantes_co = [v for v in vars_necesarias_co if v not in df.columns]
    if faltantes_co:
        print(f"  ⚠️  Variables CO faltantes: {faltantes_co}")
    else:
        print(f"  ✓ Todas las variables CO presentes")
    
    # Verificar CT
    faltantes_ct = [v for v in vars_necesarias_ct if v not in df.columns]
    if faltantes_ct:
        print(f"  ⚠️  Variables CT faltantes: {faltantes_ct}")
    else:
        print(f"  ✓ Todas las variables CT presentes")
    
    print()

Verificando variables necesarias:

Generales:
  ✓ Todas las variables CO presentes
  ✓ Todas las variables CT presentes

Ballotage:
  ✓ Todas las variables CO presentes
  ✓ Todas las variables CT presentes



## 3. Crear Variables de Congruencia e Incongruencia

In [4]:
print("Creando variables de Congruencia e Incongruencia...\n")

for nombre_df, df in dfs_Finales.items():
    print(f"Procesando {nombre_df}...")
    
    # CONGRUENTE CO = Progresistas_Izq + Conservadores_Der
    if 'Cambio_Op_Sum_Pro_Izq' in df.columns and 'Cambio_Op_Sum_Con_Der' in df.columns:
        df['CO_Congruente'] = df['Cambio_Op_Sum_Pro_Izq'] + df['Cambio_Op_Sum_Con_Der']
        print(f"  ✓ CO_Congruente creada")
    
    # INCONGRUENTE CO = Progresistas_Der + Conservadores_Izq
    if 'Cambio_Op_Sum_Pro_Der' in df.columns and 'Cambio_Op_Sum_Con_Izq' in df.columns:
        df['CO_Incongruente'] = df['Cambio_Op_Sum_Pro_Der'] + df['Cambio_Op_Sum_Con_Izq']
        print(f"  ✓ CO_Incongruente creada")
    
    # CONGRUENTE CT = Progresistas_Izq + Conservadores_Der
    if 'Cambio_Tiempo_Sum_Pro_Izq' in df.columns and 'Cambio_Tiempo_Sum_Con_Der' in df.columns:
        df['CT_Congruente'] = df['Cambio_Tiempo_Sum_Pro_Izq'] + df['Cambio_Tiempo_Sum_Con_Der']
        print(f"  ✓ CT_Congruente creada")
    
    # INCONGRUENTE CT = Progresistas_Der + Conservadores_Izq
    if 'Cambio_Tiempo_Sum_Pro_Der' in df.columns and 'Cambio_Tiempo_Sum_Con_Izq' in df.columns:
        df['CT_Incongruente'] = df['Cambio_Tiempo_Sum_Pro_Der'] + df['Cambio_Tiempo_Sum_Con_Izq']
        print(f"  ✓ CT_Incongruente creada")
    
    print()

print("✅ Variables de Congruencia/Incongruencia creadas exitosamente")

Creando variables de Congruencia e Incongruencia...

Procesando Generales...
  ✓ CO_Congruente creada
  ✓ CO_Incongruente creada
  ✓ CT_Congruente creada
  ✓ CT_Incongruente creada

Procesando Ballotage...
  ✓ CO_Congruente creada
  ✓ CO_Incongruente creada
  ✓ CT_Congruente creada
  ✓ CT_Incongruente creada

✅ Variables de Congruencia/Incongruencia creadas exitosamente


## 4. Estadísticas Descriptivas - Todas las Poblaciones

In [5]:
print("="*70)
print("ESTADÍSTICAS DESCRIPTIVAS - TODAS LAS POBLACIONES")
print("="*70)

variables_analizar = ['CO_Congruente', 'CO_Incongruente', 'CT_Congruente', 'CT_Incongruente']

for nombre_df, df in dfs_Finales.items():
    print(f"\n📊 {nombre_df}:")
    print("\n" + "-"*70)
    
    for var in variables_analizar:
        if var in df.columns:
            datos = df[var].dropna()
            
            if len(datos) > 0:
                print(f"\n{var}:")
                print(f"  n = {len(datos)}")
                print(f"  Media = {datos.mean():.4f}")
                print(f"  Mediana = {datos.median():.4f}")
                print(f"  DE = {datos.std():.4f}")
                print(f"  Min = {datos.min():.4f}")
                print(f"  Max = {datos.max():.4f}")
            else:
                print(f"\n{var}: Sin datos")
        else:
            print(f"\n{var}: Variable no encontrada")
    
    print("\n" + "-"*70)

print("\n" + "="*70)

ESTADÍSTICAS DESCRIPTIVAS - TODAS LAS POBLACIONES

📊 Generales:

----------------------------------------------------------------------

CO_Congruente:
  n = 2786
  Media = 0.7968
  Mediana = 0.0000
  DE = 3.6455
  Min = -17.0000
  Max = 19.0000

CO_Incongruente:
  n = 2786
  Media = -0.7175
  Mediana = 0.0000
  DE = 3.4926
  Min = -18.0000
  Max = 17.0000

CT_Congruente:
  n = 2786
  Media = -20.1842
  Mediana = -15.5140
  DE = 53.7236
  Min = -1268.4400
  Max = 511.7080

CT_Incongruente:
  n = 2786
  Media = -18.7598
  Mediana = -12.6457
  DE = 51.2612
  Min = -1275.3110
  Max = 495.8130

----------------------------------------------------------------------

📊 Ballotage:

----------------------------------------------------------------------

CO_Congruente:
  n = 1254
  Media = 0.3373
  Mediana = 0.0000
  DE = 3.3643
  Min = -13.0000
  Max = 18.0000

CO_Incongruente:
  n = 1254
  Media = -0.2241
  Mediana = 0.0000
  DE = 3.4411
  Min = -20.0000
  Max = 16.0000

CT_Congruente:
  n = 

## 5. Test de Wilcoxon: Congruente vs Incongruente - Todas las Poblaciones

In [6]:
print("="*70)
print("TEST DE WILCOXON PAREADO: CONGRUENTE vs INCONGRUENTE")
print("="*70)
print("\nH₀: No hay diferencia entre Congruente e Incongruente")
print("H₁: Hay diferencia entre Congruente e Incongruente\n")

resultados_wilcoxon_general = []

for nombre_df, df in dfs_Finales.items():
    print(f"\n📊 {nombre_df}:")
    print("\n" + "-"*70)
    
    # Test para CO
    if 'CO_Congruente' in df.columns and 'CO_Incongruente' in df.columns:
        # Eliminar NaN en ambas variables
        datos_pareados = df[['CO_Congruente', 'CO_Incongruente']].dropna()
        
        if len(datos_pareados) > 0:
            try:
                stat, p_valor = wilcoxon(datos_pareados['CO_Congruente'], 
                                        datos_pareados['CO_Incongruente'])
                
                # Determinar significancia
                if p_valor < 0.001:
                    sig = '***'
                elif p_valor < 0.01:
                    sig = '**'
                elif p_valor < 0.05:
                    sig = '*'
                else:
                    sig = 'ns'
                
                print(f"\nCO - Congruente vs Incongruente:")
                print(f"  n pareados = {len(datos_pareados)}")
                print(f"  Media Congruente = {datos_pareados['CO_Congruente'].mean():.4f}")
                print(f"  Media Incongruente = {datos_pareados['CO_Incongruente'].mean():.4f}")
                print(f"  Estadístico W = {stat:.4f}")
                print(f"  p-valor = {p_valor:.6f}")
                print(f"  Significancia: {sig}")
                
                if sig != 'ns':
                    print(f"  ✅ DIFERENCIA SIGNIFICATIVA")
                else:
                    print(f"  ❌ No significativa")
                
                resultados_wilcoxon_general.append({
                    'Dataset': nombre_df,
                    'Tipo': 'CO',
                    'n': len(datos_pareados),
                    'Media_Congruente': datos_pareados['CO_Congruente'].mean(),
                    'Media_Incongruente': datos_pareados['CO_Incongruente'].mean(),
                    'W': stat,
                    'p_valor': p_valor,
                    'Sig': sig
                })
                
            except Exception as e:
                print(f"\nCO - Error: {e}")
        else:
            print(f"\nCO - Sin datos pareados")
    
    # Test para CT
    if 'CT_Congruente' in df.columns and 'CT_Incongruente' in df.columns:
        # Eliminar NaN en ambas variables
        datos_pareados = df[['CT_Congruente', 'CT_Incongruente']].dropna()
        
        if len(datos_pareados) > 0:
            try:
                stat, p_valor = wilcoxon(datos_pareados['CT_Congruente'], 
                                        datos_pareados['CT_Incongruente'])
                
                # Determinar significancia
                if p_valor < 0.001:
                    sig = '***'
                elif p_valor < 0.01:
                    sig = '**'
                elif p_valor < 0.05:
                    sig = '*'
                else:
                    sig = 'ns'
                
                print(f"\nCT - Congruente vs Incongruente:")
                print(f"  n pareados = {len(datos_pareados)}")
                print(f"  Media Congruente = {datos_pareados['CT_Congruente'].mean():.4f}")
                print(f"  Media Incongruente = {datos_pareados['CT_Incongruente'].mean():.4f}")
                print(f"  Estadístico W = {stat:.4f}")
                print(f"  p-valor = {p_valor:.6f}")
                print(f"  Significancia: {sig}")
                
                if sig != 'ns':
                    print(f"  ✅ DIFERENCIA SIGNIFICATIVA")
                else:
                    print(f"  ❌ No significativa")
                
                resultados_wilcoxon_general.append({
                    'Dataset': nombre_df,
                    'Tipo': 'CT',
                    'n': len(datos_pareados),
                    'Media_Congruente': datos_pareados['CT_Congruente'].mean(),
                    'Media_Incongruente': datos_pareados['CT_Incongruente'].mean(),
                    'W': stat,
                    'p_valor': p_valor,
                    'Sig': sig
                })
                
            except Exception as e:
                print(f"\nCT - Error: {e}")
        else:
            print(f"\nCT - Sin datos pareados")
    
    print("\n" + "-"*70)

print("\n" + "="*70)

# Crear DataFrame con resultados
df_resultados_general = pd.DataFrame(resultados_wilcoxon_general)
print("\n📋 Resumen de Resultados:\n")
print(df_resultados_general.to_string(index=False))

TEST DE WILCOXON PAREADO: CONGRUENTE vs INCONGRUENTE

H₀: No hay diferencia entre Congruente e Incongruente
H₁: Hay diferencia entre Congruente e Incongruente


📊 Generales:

----------------------------------------------------------------------

CO - Congruente vs Incongruente:
  n pareados = 2786
  Media Congruente = 0.7968
  Media Incongruente = -0.7175
  Estadístico W = 651078.5000
  p-valor = 0.000000
  Significancia: ***
  ✅ DIFERENCIA SIGNIFICATIVA

CT - Congruente vs Incongruente:
  n pareados = 2786
  Media Congruente = -20.1842
  Media Incongruente = -18.7598
  Estadístico W = 1568324.5000
  p-valor = 0.000000
  Significancia: ***
  ✅ DIFERENCIA SIGNIFICATIVA

----------------------------------------------------------------------

📊 Ballotage:

----------------------------------------------------------------------

CO - Congruente vs Incongruente:
  n pareados = 1254
  Media Congruente = 0.3373
  Media Incongruente = -0.2241
  Estadístico W = 168625.0000
  p-valor = 0.009005


## 6. Análisis por Categoría (Categoria_PASO_2023)

In [7]:
print("="*70)
print("TEST DE WILCOXON POR CATEGORÍA")
print("="*70)

Categorias_Validas = [
    'Left_Wing', 'Progressivism', 'Centre',
    'Moderate_Right_A', 'Moderate_Right_B', 'Right_Wing_Libertarian'
]

Etiquetas_Categorias = {
    'Left_Wing': 'Left Wing',
    'Progressivism': 'Progressivism',
    'Centre': 'Centre',
    'Moderate_Right_A': 'Moderate Right A',
    'Moderate_Right_B': 'Moderate Right B',
    'Right_Wing_Libertarian': 'Right Wing Libertarian'
}

resultados_por_categoria = []

for nombre_df, df in dfs_Finales.items():
    print(f"\n{'='*70}")
    print(f"📊 Dataset: {nombre_df}")
    print(f"{'='*70}")
    
    for categoria in Categorias_Validas:
        print(f"\n{Etiquetas_Categorias[categoria]}:")
        print("-"*70)
        
        # Filtrar por categoría
        df_cat = df[df['Categoria_PASO_2023'] == categoria].copy()
        
        # Test para CO
        if 'CO_Congruente' in df_cat.columns and 'CO_Incongruente' in df_cat.columns:
            datos_pareados = df_cat[['CO_Congruente', 'CO_Incongruente']].dropna()
            
            if len(datos_pareados) > 1:
                try:
                    stat, p_valor = wilcoxon(datos_pareados['CO_Congruente'], 
                                            datos_pareados['CO_Incongruente'])
                    
                    if p_valor < 0.001:
                        sig = '***'
                    elif p_valor < 0.01:
                        sig = '**'
                    elif p_valor < 0.05:
                        sig = '*'
                    else:
                        sig = 'ns'
                    
                    print(f"\n  CO:")
                    print(f"    n = {len(datos_pareados)}")
                    print(f"    Media Congruente = {datos_pareados['CO_Congruente'].mean():.4f}")
                    print(f"    Media Incongruente = {datos_pareados['CO_Incongruente'].mean():.4f}")
                    print(f"    p-valor = {p_valor:.6f}")
                    print(f"    Sig: {sig} {'✅' if sig != 'ns' else '❌'}")
                    
                    resultados_por_categoria.append({
                        'Dataset': nombre_df,
                        'Categoria': categoria,
                        'Etiqueta': Etiquetas_Categorias[categoria],
                        'Tipo': 'CO',
                        'n': len(datos_pareados),
                        'Media_Congruente': datos_pareados['CO_Congruente'].mean(),
                        'Media_Incongruente': datos_pareados['CO_Incongruente'].mean(),
                        'p_valor': p_valor,
                        'Sig': sig
                    })
                    
                except Exception as e:
                    print(f"\n  CO: Error - {e}")
            else:
                print(f"\n  CO: Datos insuficientes (n={len(datos_pareados)})")
        
        # Test para CT
        if 'CT_Congruente' in df_cat.columns and 'CT_Incongruente' in df_cat.columns:
            datos_pareados = df_cat[['CT_Congruente', 'CT_Incongruente']].dropna()
            
            if len(datos_pareados) > 1:
                try:
                    stat, p_valor = wilcoxon(datos_pareados['CT_Congruente'], 
                                            datos_pareados['CT_Incongruente'])
                    
                    if p_valor < 0.001:
                        sig = '***'
                    elif p_valor < 0.01:
                        sig = '**'
                    elif p_valor < 0.05:
                        sig = '*'
                    else:
                        sig = 'ns'
                    
                    print(f"\n  CT:")
                    print(f"    n = {len(datos_pareados)}")
                    print(f"    Media Congruente = {datos_pareados['CT_Congruente'].mean():.4f}")
                    print(f"    Media Incongruente = {datos_pareados['CT_Incongruente'].mean():.4f}")
                    print(f"    p-valor = {p_valor:.6f}")
                    print(f"    Sig: {sig} {'✅' if sig != 'ns' else '❌'}")
                    
                    resultados_por_categoria.append({
                        'Dataset': nombre_df,
                        'Categoria': categoria,
                        'Etiqueta': Etiquetas_Categorias[categoria],
                        'Tipo': 'CT',
                        'n': len(datos_pareados),
                        'Media_Congruente': datos_pareados['CT_Congruente'].mean(),
                        'Media_Incongruente': datos_pareados['CT_Incongruente'].mean(),
                        'p_valor': p_valor,
                        'Sig': sig
                    })
                    
                except Exception as e:
                    print(f"\n  CT: Error - {e}")
            else:
                print(f"\n  CT: Datos insuficientes (n={len(datos_pareados)})")

print("\n" + "="*70)

# Crear DataFrame con resultados por categoría
df_resultados_categoria = pd.DataFrame(resultados_por_categoria)
print("\n📋 Resumen de Resultados por Categoría:\n")
if len(df_resultados_categoria) > 0:
    print(df_resultados_categoria[['Dataset', 'Etiqueta', 'Tipo', 'n', 'Media_Congruente', 
                                   'Media_Incongruente', 'p_valor', 'Sig']].to_string(index=False))
else:
    print("  No hay resultados")

TEST DE WILCOXON POR CATEGORÍA

📊 Dataset: Generales

Left Wing:
----------------------------------------------------------------------

  CO:
    n = 425
    Media Congruente = 0.8541
    Media Incongruente = -0.7529
    p-valor = 0.000000
    Sig: *** ✅

  CT:
    n = 425
    Media Congruente = -16.0042
    Media Incongruente = -14.7747
    p-valor = 0.000000
    Sig: *** ✅

Progressivism:
----------------------------------------------------------------------

  CO:
    n = 1257
    Media Congruente = 0.9666
    Media Incongruente = -0.7765
    p-valor = 0.000000
    Sig: *** ✅

  CT:
    n = 1257
    Media Congruente = -19.4113
    Media Incongruente = -17.9198
    p-valor = 0.000000
    Sig: *** ✅

Centre:
----------------------------------------------------------------------

  CO:
    n = 84
    Media Congruente = 0.1726
    Media Incongruente = -0.4940
    p-valor = 0.667781
    Sig: ns ❌

  CT:
    n = 84
    Media Congruente = -20.6898
    Media Incongruente = -19.1716
    p-v

## 7. Resumen de Resultados Significativos

In [8]:
print("="*70)
print("RESUMEN DE RESULTADOS SIGNIFICATIVOS")
print("="*70)

print("\n📊 Análisis General (Todas las Poblaciones):")
print("-"*70)
if len(df_resultados_general) > 0:
    sig_general = df_resultados_general[df_resultados_general['Sig'] != 'ns']
    print(f"\nTotal de comparaciones: {len(df_resultados_general)}")
    print(f"Comparaciones significativas: {len(sig_general)}")
    if len(sig_general) > 0:
        print("\nResultados significativos:")
        for idx, row in sig_general.iterrows():
            print(f"  ✅ {row['Dataset']} - {row['Tipo']}: p={row['p_valor']:.6f} ({row['Sig']})")
    else:
        print("\n  ❌ No hay comparaciones significativas")
else:
    print("  No hay resultados")

print("\n📊 Análisis por Categoría:")
print("-"*70)
if len(df_resultados_categoria) > 0:
    sig_categoria = df_resultados_categoria[df_resultados_categoria['Sig'] != 'ns']
    print(f"\nTotal de comparaciones: {len(df_resultados_categoria)}")
    print(f"Comparaciones significativas: {len(sig_categoria)}")
    
    if len(sig_categoria) > 0:
        print("\nResultados significativos por Dataset:")
        for dataset in df_resultados_categoria['Dataset'].unique():
            print(f"\n  {dataset}:")
            sig_dataset = sig_categoria[sig_categoria['Dataset'] == dataset]
            if len(sig_dataset) > 0:
                for idx, row in sig_dataset.iterrows():
                    print(f"    ✅ {row['Etiqueta']} - {row['Tipo']}: p={row['p_valor']:.6f} ({row['Sig']})")
            else:
                print(f"    ❌ Sin resultados significativos")
    else:
        print("\n  ❌ No hay comparaciones significativas por categoría")
else:
    print("  No hay resultados")

print("\n" + "="*70)

RESUMEN DE RESULTADOS SIGNIFICATIVOS

📊 Análisis General (Todas las Poblaciones):
----------------------------------------------------------------------

Total de comparaciones: 4
Comparaciones significativas: 4

Resultados significativos:
  ✅ Generales - CO: p=0.000000 (***)
  ✅ Generales - CT: p=0.000000 (***)
  ✅ Ballotage - CO: p=0.009005 (**)
  ✅ Ballotage - CT: p=0.000006 (***)

📊 Análisis por Categoría:
----------------------------------------------------------------------

Total de comparaciones: 24
Comparaciones significativas: 11

Resultados significativos por Dataset:

  Generales:
    ✅ Left Wing - CO: p=0.000000 (***)
    ✅ Left Wing - CT: p=0.000000 (***)
    ✅ Progressivism - CO: p=0.000000 (***)
    ✅ Progressivism - CT: p=0.000000 (***)
    ✅ Moderate Right A - CO: p=0.017383 (*)
    ✅ Moderate Right B - CO: p=0.010212 (*)
    ✅ Right Wing Libertarian - CO: p=0.002280 (**)
    ✅ Right Wing Libertarian - CT: p=0.004164 (**)

  Ballotage:
    ✅ Progressivism - CO: p=0.00

## 8. Guardar Resultados en Excel

In [9]:
# Guardar en la carpeta de procesados
Carpeta_Salida = os.path.join(os.getcwd(), '..', 'Data', 'Procesados')
if not os.path.exists(Carpeta_Salida):
    os.makedirs(Carpeta_Salida)

# Guardar bases actualizadas con las nuevas variables
for nombre_df, df in dfs_Finales.items():
    archivo = f'{nombre_df}_con_Congruencia.xlsx'
    ruta = os.path.join(Carpeta_Salida, archivo)
    df.to_excel(ruta, index=False)
    print(f"✓ {archivo} guardado")

# Guardar resultados
if len(df_resultados_general) > 0:
    ruta_resultados_general = os.path.join(Carpeta_Salida, 'Resultados_Congruencia_General.xlsx')
    df_resultados_general.to_excel(ruta_resultados_general, index=False)
    print(f"\n✓ Resultados generales guardados en: {ruta_resultados_general}")

if len(df_resultados_categoria) > 0:
    ruta_resultados_categoria = os.path.join(Carpeta_Salida, 'Resultados_Congruencia_Por_Categoria.xlsx')
    df_resultados_categoria.to_excel(ruta_resultados_categoria, index=False)
    print(f"✓ Resultados por categoría guardados en: {ruta_resultados_categoria}")

print("\n✅ Todos los archivos guardados exitosamente")

✓ Generales_con_Congruencia.xlsx guardado
✓ Ballotage_con_Congruencia.xlsx guardado

✓ Resultados generales guardados en: c:\Users\Patricio\Documents\Codigo\Python\Investigacion\Tesis\Código\..\Data\Procesados\Resultados_Congruencia_General.xlsx
✓ Resultados por categoría guardados en: c:\Users\Patricio\Documents\Codigo\Python\Investigacion\Tesis\Código\..\Data\Procesados\Resultados_Congruencia_Por_Categoria.xlsx

✅ Todos los archivos guardados exitosamente


## 9. Resumen Final

In [10]:
print("="*70)
print("RESUMEN FINAL: ANÁLISIS DE CONGRUENCIA IDEOLÓGICA")
print("="*70)

print("\n📊 Variables creadas:")
print("  - CO_Congruente (Progresistas_Izq + Conservadores_Der)")
print("  - CO_Incongruente (Progresistas_Der + Conservadores_Izq)")
print("  - CT_Congruente (Progresistas_Izq + Conservadores_Der)")
print("  - CT_Incongruente (Progresistas_Der + Conservadores_Izq)")

print("\n📈 Análisis realizados:")
print("  1. Estadísticas descriptivas (todas las poblaciones)")
print("  2. Test de Wilcoxon pareado (todas las poblaciones)")
print("  3. Test de Wilcoxon pareado por categoría")
print("  4. Para Generales y Ballotage")

print("\n📁 Archivos generados:")
print("  - Generales_con_Congruencia.xlsx")
print("  - Ballotage_con_Congruencia.xlsx")
print("  - Resultados_Congruencia_General.xlsx")
print("  - Resultados_Congruencia_Por_Categoria.xlsx")

print("\n🎯 Interpretación:")
print("  - Congruente > Incongruente: Más cambios ideológicamente consistentes")
print("  - Congruente < Incongruente: Más cambios ideológicamente inconsistentes")
print("  - p < 0.05: La diferencia es estadísticamente significativa")

print("\n" + "="*70)
print("✅ ANÁLISIS COMPLETADO")
print("="*70)

RESUMEN FINAL: ANÁLISIS DE CONGRUENCIA IDEOLÓGICA

📊 Variables creadas:
  - CO_Congruente (Progresistas_Izq + Conservadores_Der)
  - CO_Incongruente (Progresistas_Der + Conservadores_Izq)
  - CT_Congruente (Progresistas_Izq + Conservadores_Der)
  - CT_Incongruente (Progresistas_Der + Conservadores_Izq)

📈 Análisis realizados:
  1. Estadísticas descriptivas (todas las poblaciones)
  2. Test de Wilcoxon pareado (todas las poblaciones)
  3. Test de Wilcoxon pareado por categoría
  4. Para Generales y Ballotage

📁 Archivos generados:
  - Generales_con_Congruencia.xlsx
  - Ballotage_con_Congruencia.xlsx
  - Resultados_Congruencia_General.xlsx
  - Resultados_Congruencia_Por_Categoria.xlsx

🎯 Interpretación:
  - Congruente > Incongruente: Más cambios ideológicamente consistentes
  - Congruente < Incongruente: Más cambios ideológicamente inconsistentes
  - p < 0.05: La diferencia es estadísticamente significativa

✅ ANÁLISIS COMPLETADO
